# Wrangle Exercises

These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

## Import Libraries

In [158]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import *
import datetime

spark = SparkSession.builder.getOrCreate()

## Part 1, Data Acquisition

### Read the case, department, and source data into their own spark dataframes

In [39]:
case_df = (
    spark.read.format("csv")
    .option("sep", ",")
    .option("inferSchema", True)
    .option("header", True)
    .load("case.csv")
)

In [40]:
dept_df = (
    spark.read.format("csv")
    .option("sep", ",")
    .option("inferSchema", True)
    .option("header", True)
    .load("dept.csv")
)

In [78]:
source_df = (
    spark.read.format("csv")
    .option("sep", ",")
    .option("inferSchema", True)
    .option("header", True)
    .load("source.csv")
)

In [42]:
case_df.show(5, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [43]:
dept_df.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



In [10]:
source_df.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



### Let's see how writing to the local disk works in spark:

#### Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [27]:
(
    case_df.write.format("csv")
    .mode("overwrite")
    .option("header", "true")
    .save("case_csv")
)

In [28]:
(
    case_df.write.format("json")
    .mode("overwrite")
    .option("header", "true")
    .save("case_json")
)

In [29]:
(
    dept_df.write.format("csv")
    .mode("overwrite")
    .option("header", "true")
    .save("dept_csv")
)

In [30]:
(
    dept_df.write.format("json")
    .mode("overwrite")
    .option("header", "true")
    .save("dept_csv")
)

In [31]:
(
    source_df.write.format("csv")
    .mode("overwrite")
    .option("header", "true")
    .save("source_csv")
)

In [32]:
(
    source_df.write.format("json")
    .mode("overwrite")
    .option("header", "true")
    .save("source_csv")
)

#### Inspect your folder structure. What do you notice?

The folders each have many individual csv and json files. 

### Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

#### Case 

In [44]:
case_df.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

The data values should be in date format. The case closed column could also be a boolean.

In [45]:
fmt = "M/d/yy H:mm"
case_df = (
    case_df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("SLA_due_date", to_timestamp("SLA_due_date", fmt))
)

In [91]:
case_df = case_df.withColumn("case_closed",col("case_closed").cast(BooleanType())) 

In [92]:
case_df.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [93]:
case_df.show(5, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03 08:11:00  
 SLA_due_date         | 2018-01-05 08:30:00  
 case_late            | NO                   
 num_days_late        | -2.0126041

#### Department

In [48]:
dept_df.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [49]:
dept_df.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



The `dept_subject_to_SLA` column could be a boolean. 

In [60]:
dept_df = dept_df.withColumn("dept_subject_to_SLA",col("dept_subject_to_SLA").cast(BooleanType())) 

In [61]:
dept_df.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|               true|
|               Brush|Solid Waste Manag...|           Solid Waste|               true|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|               true|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|               true|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|               true|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



#### Source

In [79]:
source_df.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [80]:
source_df.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



The `source_id` could be an integer value. 

In [82]:
source_df = source_df.withColumn("source_id", source_df.source_id.cast(IntegerType()))

In [83]:
source_df.dtypes

[('source_id', 'int'), ('source_username', 'string')]

In [84]:
source_df.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



## Part 2

### How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

First we must make a subset of the current open issues, then find the SLA date furthest in the past.

In [88]:
case_df.show(2, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03 08:11:00  
 SLA_due_date         | 2018-01-05 08:30:00  
 case_late            | NO                   
 num_days_late        | -2.0126041

In [105]:
case_df = (
    case_df.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

open_cases = case_df.select(
    "case_closed",
    "case_opened_date",
    "SLA_due_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("! case_closed"))

In [106]:
open_cases.show(10)

+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|       SLA_due_date|case_age|days_to_closed|case_lifetime|
+-----------+-------------------+-------------------+--------+--------------+-------------+
|      false|2018-01-02 09:39:00|2018-01-09 09:39:00|    1406|          null|         1406|
|      false|2018-01-02 10:49:00|2018-05-10 10:49:00|    1406|          null|         1406|
|      false|2018-01-02 13:45:00|2018-03-07 13:45:00|    1406|          null|         1406|
|      false|2018-01-02 14:09:00|2018-05-10 14:09:00|    1406|          null|         1406|
|      false|2018-01-02 14:34:00|2018-05-10 14:34:00|    1406|          null|         1406|
|      false|2018-01-02 15:22:00|2018-01-09 15:22:00|    1406|          null|         1406|
|      false|2018-01-02 15:58:00|2018-01-17 15:58:00|    1406|          null|         1406|
|      false|2018-01-03 08:04:00|2018-01-10 08:30:00|    1405|          null|   

In [127]:
open_cases.select(min("case_opened_date")).show()

+---------------------+
|min(case_opened_date)|
+---------------------+
|  2017-01-01 13:48:00|
+---------------------+



In [128]:
min_open_date = '2017-01-01 13:48:00'

In [124]:
open_cases.select(min("SLA_due_date")).show()

+-------------------+
|  min(SLA_due_date)|
+-------------------+
|2017-01-17 08:30:00|
+-------------------+



In [132]:
min_SLA_date = "2017-01-17 08:30:00"

In [130]:
case_df.where(col("case_opened_date") == min_open_date).show(vertical = True)

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | null                 
 SLA_due_date         | 2017-01-17 08:30:00  
 case_late            | YES                  
 num_days_late        | 348.6458333          
 case_closed          | false                
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 SLA_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 case_age             | 1772                 
 days_to_closed       | null                 
 case_lifetime        | 1772                 



In [134]:
case_df.where((col("SLA_due_date") == min_SLA_date) & (expr("! case_closed")) ).show(vertical = True)

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | null                 
 SLA_due_date         | 2017-01-17 08:30:00  
 case_late            | YES                  
 num_days_late        | 348.6458333          
 case_closed          | false                
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 SLA_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 case_age             | 1772                 
 days_to_closed       | null                 
 case_lifetime        | 1772                 
-RECORD 1------------------------------------
 case_id              | 1013225651           
 case_opened_date     | 2017-01-01 13:57:00  
 case_closed_date     | null      

### How many Stray Animal cases are there?

In [135]:
stray_animals_cases = case_df.where(col('service_request_type') == "Stray Animal")

In [137]:
stray_animals_cases.show(5, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1407                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
-RECORD 1------------------------------------
 case_id              | 1014127380           
 case_opened_date     | 2018-01-01 08:39:00  
 case_closed_date     | 2018-01-01

In [141]:
stray_animals_cases.count()

26760

There are 26,760 cases of stray animals.

### How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [149]:
field_not_standby = case_df.where((col('dept_division') == 'Field Operations')&(col('service_request_type') != 'Officer Standby'))

In [152]:
field_not_standby.show(2, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1407                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
-RECORD 1------------------------------------
 case_id              | 1014127336           
 case_opened_date     | 2018-01-01 01:34:00  
 case_closed_date     | 2018-01-01

In [153]:
field_not_standby.count()

113902

There are 113,902 service requests in the Field Operations departments not classified as Officer Standby. 

### Convert the council_district column to a string column.

In [ ]:
source_df = source_df.withColumn("source_id", source_df.source_id.cast(IntegerType()))

In [154]:
case_df = case_df.withColumn('council_district', case_df.council_district.cast(StringType()))

In [155]:
case_df.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int')]

In [156]:
case_df.show(2, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1407                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03

### Extract the year from the case_closed_date column.

In [160]:
case_df.select(year("case_opened_date")).show()

+----------------------+
|year(case_opened_date)|
+----------------------+
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
+----------------------+
only showing top 20 rows



### Convert num_days_late from days to hours in new columns num_hours_late.


In [161]:
case_df.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int')]

In [176]:
case_df.withColumn("num_hours_late", col("num_days_late") * 24).show(2, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1407                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 num_hours_late       | -23964.2102784       
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01

### Join the case data with the source and department data.

In [177]:
case_df.show(1, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1407                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
only showing top 1 row



In [178]:
dept_df.show(1, vertical = True)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | true             
only showing top 1 row



In [179]:
source_df.show(1, vertical = True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



In [182]:
df = (
    case_df
    # left join on dept_division
    .join(dept_df, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept_df.dept_division)
    .drop(dept_df.dept_name)
    .drop(dept_df.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)


In [184]:
df.show(2, vertical = True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1407                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
-RECORD 1------------------------------------
 dept_division        | Storm Wate

In [185]:
df = (
    df
    # left join on dept_division
    .join(source_df, "source_id", "left")
)

In [186]:
df.show(2, vertical = True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1407                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 source_username      | null                 
-RECORD 1-------------------------

### Are there any cases that do not have a request source?

In [222]:
df.filter("source_id is null").count()

0

There are no values that have no source id. 

In [223]:
df.filter("source_username is null").count()

604269

There are, however, 604,269 cases that lack a request source. 

### What are the top 10 service request types in terms of number of requests?

In [246]:
df.groupby('service_request_type').count().sort(desc('count')).show(10, False)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



### What are the top 10 service request types in terms of average days late?

In [253]:
(
df
    .groupby('service_request_type')
    .agg(avg(col('num_days_late')))
    .sort(desc('avg(num_days_late)'))
    .show(10, False)
    
)

+--------------------------------------+------------------+
|service_request_type                  |avg(num_days_late)|
+--------------------------------------+------------------+
|Zoning: Junk Yards                    |175.95636210420946|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Storage of Used Mattress              |142.11255641500003|
|Zoning: Recycle Yard                  |135.928516124798  |
|Donation Container Enforcement        |131.75610506358706|
|License Requied Used Mattress Sales   |128.79828704142858|
|Traffic Signal Graffiti               |101.79846062200002|
|Complaint                             |72.87050230311695 |
+--------------------------------------+------------------+
only showing top 10 rows



### Does number of days late depend on department?

In [259]:
(
df
    .groupby('department')
    .agg(avg(col('num_days_late')))
    .sort(desc('avg(num_days_late)'))
    .show(10, False)
    
)

+------------------------+-------------------+
|department              |avg(num_days_late) |
+------------------------+-------------------+
|Customer Service        |59.7370914963008   |
|Solid Waste             |-2.200057513672157 |
|Metro Health            |-4.911766979607001 |
|Parks & Recreation      |-5.251521960055148 |
|Trans & Cap Improvements|-20.612837354052704|
|DSD/Code Enforcement    |-38.36938892614494 |
|Animal Care Services    |-226.51783940550297|
|City Council            |null               |
+------------------------+-------------------+



Yes. Customer service is the only department that on average is late. 

### How do number of days late depend on department and request type?

In [260]:
(
df
    .groupby('department', 'service_request_type')
    .agg(avg(col('num_days_late')))
    .sort(desc('avg(num_days_late)'))
    .show(20, False)
    
)

+------------------------+--------------------------------------+------------------+
|department              |service_request_type                  |avg(num_days_late)|
+------------------------+--------------------------------------+------------------+
|DSD/Code Enforcement    |Zoning: Junk Yards                    |175.95636210420946|
|DSD/Code Enforcement    |Labeling for Used Mattress            |162.43032902285717|
|DSD/Code Enforcement    |Record Keeping of Used Mattresses     |153.99724039428568|
|DSD/Code Enforcement    |Signage Requied for Sale of Used Mattr|151.63868055333333|
|DSD/Code Enforcement    |Storage of Used Mattress              |142.11255641500003|
|DSD/Code Enforcement    |Zoning: Recycle Yard                  |135.928516124798  |
|DSD/Code Enforcement    |Donation Container Enforcement        |131.75610506358706|
|DSD/Code Enforcement    |License Requied Used Mattress Sales   |128.79828704142858|
|Trans & Cap Improvements|Traffic Signal Graffiti               |

### You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.

In [263]:
df.show(2, vertical = True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1407                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 source_username      | null                 
-RECORD 1-------------------------

In [267]:
df.dtypes

[('source_id', 'string'),
 ('dept_division', 'string'),
 ('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int'),
 ('department', 'string'),
 ('dept_subject_to_SLA', 'boolean'),
 ('source_username', 'string')]

In [266]:
df.select(max("case_opened_date")).show()

+---------------------+
|max(case_opened_date)|
+---------------------+
|  2018-08-08 10:38:00|
+---------------------+



In [289]:
df.select(max("case_closed_date")).show()

+---------------------+
|max(case_closed_date)|
+---------------------+
|  2018-08-08 10:38:00|
+---------------------+



In [329]:
df = df.withColumn("current_time", lit('2018-08-08 15:07:00'))

In [330]:
df.show(1, vertical = True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1407                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 source_username      | null                 
 current_time         | 2018-08-08

In [332]:
df = df.withColumn('current_time', to_timestamp('current_time'))

In [333]:
df.show(1, vertical = True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1407                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 source_username      | null                 
 current_time         | 2018-08-08

In [334]:
df = (
    df.withColumn(
        "case_age", datediff('current_time', "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)


In [335]:
df.show(1, vertical = True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 219                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 source_username      | null                 
 current_time         | 2018-08-08

In [336]:
df = case_df.drop('current_time')

In [339]:
case_df.show(5, vertical = True, truncate = False)

-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 2018-01-01 00:42:00                   
 case_closed_date     | 2018-01-01 12:29:00                   
 SLA_due_date         | 2020-09-26 00:42:00                   
 case_late            | NO                                    
 num_days_late        | -998.5087616000001                    
 case_closed          | true                                  
 dept_division        | Field Operations                      
 service_request_type | Stray Animal                          
 SLA_days             | 999.0                                 
 case_status          | Closed                                
 source_id            | svcCRMLS                              
 request_address      | 2315  EL PASO ST, San Antonio, 78207  
 council_district     | 5                                     
 case_age             | 219                            